In [2]:
import glob
import os
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

In [35]:
df = pd.read_csv("data.csv")

In [37]:
df.head

<bound method NDFrame.head of            a1    b1    c1    d1    e1    f1 g1    h1 a2    b2  ... to_h7  \
0           R     N     B     Q     K     B  N     R  P     P  ...     0   
1           R     N     B     Q     K     B  N     R  P     P  ...     0   
2           R     N     B     Q     K     B  N     R  P     P  ...     0   
3           R     N     B     Q     K     B  N     R  P     P  ...     0   
4           R     N     B     Q     K     B  N     R  P     P  ...     0   
...       ...   ...   ...   ...   ...   ... ..   ... ..   ...  ...   ...   
1685519  None  None  None  None  None  None  K  None  P  None  ...     0   
1685520  None  None  None  None  None  None  K  None  P  None  ...     0   
1685521  None  None  None  None  None  None  K  None  P  None  ...     0   
1685522  None  None  None  None  None  None  K  None  P  None  ...     0   
1685523  None  None  None  None  None  None  K  None  P  None  ...     0   

        to_a8 to_b8 to_c8 to_d8 to_e8 to_f8 to_g8 to_h8 g

In [38]:
train = shuffle(df)

In [39]:
train.head()

,a1,b1,c1,d1,e1,f1,g1,h1,a2,b2,...,to_h7,to_a8,to_b8,to_c8,to_d8,to_e8,to_f8,to_g8,to_h8,good_move
1642325,None,None,None,None,None,None,K,None,P,P,...,0,0,0,0,0,0,0,0,0,0
21819,None,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
549711,None,B,R,R,None,None,K,None,P,P,...,0,0,0,0,0,0,0,0,0,0
150623,None,None,None,None,R,None,K,None,None,None,...,0,0,1,0,0,0,0,0,0,0
96018,None,None,None,None,None,None,None,None,B,None,...,0,0,0,1,0,0,0,0,0,0


In [7]:
features = list(train.iloc[:, 0:192].columns)

In [40]:
X = train[features]
y = train['good_move']

In [41]:
categorical_columns = list(X.iloc[:, 0:63].columns)
numerical_columns = list(X.iloc[:, 64:192].columns)
feature_columns = []

for feature_name in categorical_columns:
    vocabulary = X[feature_name].unique()
    feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))


for feature_name in numerical_columns:
    feature_columns.append(tf.feature_column.numeric_column(feature_name,dtype = tf.float32))

In [42]:
def make_input_fn(data_df, label_df, num_epochs = 10, shuffle = True, batch_size = 32):
    def input_function():
        ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
        if shuffle:
            ds = ds.shuffle(1000)
        ds = ds.batch(batch_size).repeat(num_epochs)
        return ds
    return input_function

In [43]:
def split_into_batches(df, batch_size=100000):
    nb_rows = len(df.index)
    intervals = []
    
    for i in range(0, nb_rows + 1, batch_size):
        intervals.append(i)
  
    if(intervals[-1] != nb_rows):
        intervals.append(nb_rows)
  
    batches_X = []
    batches_y = []
  
    for i in range(0, len(intervals) - 1):
        batches_X.append(df.iloc[intervals[i]:intervals[i + 1], :][features])
        batches_y.append(df.iloc[intervals[i]:intervals[i + 1], :]['good_move'])

    return batches_X, batches_y

In [44]:
batches_X, batches_y = split_into_batches(train)

In [45]:
type(train)

pandas.core.frame.DataFrame

In [46]:
##initialize a classifier
linear_est = tf.estimator.LinearClassifier(feature_columns = feature_columns, model_dir='\model')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '\\model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0xb9d9cf5f8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [47]:
input_functions = []
for df_X, df_y in zip(batches_X, batches_y):
    input_functions.append(make_input_fn(df_X, df_y))

In [48]:
# train the model on all the input functions
i = 1
for input_function in input_functions:
    print('<======================================== NEW BATCH ========================================>')
    print('Batch: ' + str(i))
    i = i + 1
    linear_est.train(input_function)
  
 # save the model
serving_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(
  tf.feature_column.make_parse_example_spec(feature_columns))

estimator_base_path = 'model'
estimator_path = linear_est.export_saved_model(estimator_base_path, serving_input_fn)

<======================================== NEW BATCH ========================================>
Batch: 1
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into \model/model.ckpt.
INFO:tensorflow:loss = 0.6931472, step = 0
INFO:tensorflow:global_step/sec: 4.05814
INFO:tensorflow:loss = 0.22562647, step = 100 (24.596 sec)
INFO:tensorflow:global_step/sec: 25.4518
INFO:tensorflow:loss = 0.25872242, step = 200 (3.922 sec)
INFO:tensorflow:global_step/sec: 26.2616
INFO:tensorflow:loss = 0.17813843, step = 300 (3.807 sec)
INFO:tensorflow:global_step/sec: 24.4702
INFO:tensorflow:loss = 0.14249283, step = 400 (4.086 sec)
INFO:tensorflow:global_step/sec: 23.3585
INFO:tensorflow:loss = 0.1479413, step = 500 (4.283 sec)
INFO:tensorflow:global_step/sec: 23.786
INFO:tensorflow:l

INFO:tensorflow:global_step/sec: 17.8692
INFO:tensorflow:loss = 0.013862126, step = 7900 (5.596 sec)
INFO:tensorflow:global_step/sec: 21.8073
INFO:tensorflow:loss = 0.21291335, step = 8000 (4.587 sec)
INFO:tensorflow:global_step/sec: 20.6509
INFO:tensorflow:loss = 0.26674348, step = 8100 (4.852 sec)
INFO:tensorflow:global_step/sec: 19.5762
INFO:tensorflow:loss = 0.12644872, step = 8200 (5.098 sec)
INFO:tensorflow:global_step/sec: 18.8572
INFO:tensorflow:loss = 0.044337586, step = 8300 (5.303 sec)
INFO:tensorflow:global_step/sec: 17.7529
INFO:tensorflow:loss = 0.11210169, step = 8400 (5.635 sec)
INFO:tensorflow:global_step/sec: 24.3927
INFO:tensorflow:loss = 0.026074126, step = 8500 (4.098 sec)
INFO:tensorflow:global_step/sec: 22.2837
INFO:tensorflow:loss = 0.144597, step = 8600 (4.487 sec)
INFO:tensorflow:global_step/sec: 15.923
INFO:tensorflow:loss = 0.16880476, step = 8700 (6.280 sec)
INFO:tensorflow:global_step/sec: 21.5293
INFO:tensorflow:loss = 0.114220805, step = 8800 (4.645 sec)

INFO:tensorflow:global_step/sec: 25.112
INFO:tensorflow:loss = 0.037052028, step = 16000 (3.982 sec)
INFO:tensorflow:global_step/sec: 25.6139
INFO:tensorflow:loss = 0.11984097, step = 16100 (3.904 sec)
INFO:tensorflow:global_step/sec: 26.2571
INFO:tensorflow:loss = 0.1164703, step = 16200 (3.809 sec)
INFO:tensorflow:global_step/sec: 25.1407
INFO:tensorflow:loss = 0.15545294, step = 16300 (3.977 sec)
INFO:tensorflow:global_step/sec: 25.6672
INFO:tensorflow:loss = 0.1503754, step = 16400 (3.896 sec)
INFO:tensorflow:global_step/sec: 25.9566
INFO:tensorflow:loss = 0.024061792, step = 16500 (3.852 sec)
INFO:tensorflow:global_step/sec: 25.4678
INFO:tensorflow:loss = 0.10589732, step = 16600 (3.927 sec)
INFO:tensorflow:global_step/sec: 25.3955
INFO:tensorflow:loss = 0.035391532, step = 16700 (3.939 sec)
INFO:tensorflow:global_step/sec: 25.7469
INFO:tensorflow:loss = 0.23690102, step = 16800 (3.883 sec)
INFO:tensorflow:global_step/sec: 26.2647
INFO:tensorflow:loss = 0.112676345, step = 16900 (

INFO:tensorflow:loss = 0.032458022, step = 24000 (3.940 sec)
INFO:tensorflow:global_step/sec: 24.884
INFO:tensorflow:loss = 0.031570505, step = 24100 (4.015 sec)
INFO:tensorflow:global_step/sec: 26.2242
INFO:tensorflow:loss = 0.12235224, step = 24200 (3.813 sec)
INFO:tensorflow:global_step/sec: 26.3672
INFO:tensorflow:loss = 0.24449356, step = 24300 (3.791 sec)
INFO:tensorflow:global_step/sec: 25.5017
INFO:tensorflow:loss = 0.23419061, step = 24400 (3.922 sec)
INFO:tensorflow:global_step/sec: 25.2214
INFO:tensorflow:loss = 0.15667258, step = 24500 (3.965 sec)
INFO:tensorflow:global_step/sec: 26.1208
INFO:tensorflow:loss = 0.04038237, step = 24600 (3.831 sec)
INFO:tensorflow:global_step/sec: 26.3362
INFO:tensorflow:loss = 0.23758996, step = 24700 (3.795 sec)
INFO:tensorflow:global_step/sec: 25.2168
INFO:tensorflow:loss = 0.09633818, step = 24800 (3.965 sec)
INFO:tensorflow:global_step/sec: 25.2782
INFO:tensorflow:loss = 0.0296812, step = 24900 (3.956 sec)
INFO:tensorflow:global_step/sec

ValueError: The passed save_path is not a valid checkpoint: \model/model.ckpt-31250